In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
import toml
import re
from zipfile import ZipFile
import urllib
from datetime import datetime
import string
import pygsheets
import requests
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import Bio.Entrez as Entrez
import benchlingapi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.benchling as bapi
from paulssonlab.api.util import base_url
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.design as design
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.viennarna as viennarna
import paulssonlab.cloning.thermodynamics as thermodynamics
import paulssonlab.cloning.primers as primers
import paulssonlab.cloning.ncbi as ncbi
import paulssonlab.cloning.commands.parser as cmd_parser
import paulssonlab.cloning.commands.semantics as cmd_semantics

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

In [ ]:
olib_oligos = reg[("oLIB", "oligos")]
olt_oligos = reg[("oLT", "oligos")]
plib_plasmids = reg[("pLIB", "plasmids")]
plib_maps = reg[("pLIB", "maps")]
lib_parts = reg[("LIB", "parts")]
part_types = reg[("LIB", "parts", "Part types")]

# Config

In [ ]:
# tags
tag_part_type = "ClpXP_tag"
tag_overhangs = workflow.overhangs_for(part_types[tag_part_type])
library_primer = "oLIB46"
tag_enzyme = Restriction.BsaI
placeholder_enzyme = Restriction.BsaI
num_random_bases = 6  # random bases to add between enzyme binding site and end of DNA
clpx_tag_length = 3  # aa
nnk_length = 6  # aa
nnk_clpx_length = nnk_length + clpx_tag_length  # aa
head_length = 6  # nt
tail_length = clpx_tag_length * 3  # nt
part_tags = "degradation clpxp sequestration"
reference = "Andersen, J. B., Sternberg, C., Poulsen, L. K., Bjørn, S. P., Givskov, M., & Molin, S. (1998). New unstable variants of green fluorescent protein for studies of transient gene expression in bacteria. Applied and environmental microbiology, 64(6), 2240-2246."

# placeholders
tm_binding = 60
tm_homology = 55
min_mfe = None

# Deg tags

## Data

In [ ]:
# FROM: Andersen, J. B., Sternberg, C., Poulsen, L. K., Bjørn, S. P., Givskov, M., & Molin, S. (1998). New unstable variants of green fluorescent protein for studies of transient gene expression in bacteria. Applied and environmental microbiology, 64(6), 2240-2246.
# paper gives reverse-complement sequences
tags_wt_rc = {
    "LAA": "AGCTGCTAAAGCGTAGTTTTCGTCGTTTGCTGC",
    "AAV": "AACTGCTGCAGCGTAGTTTTCGTCGTTTGCTGC",
    # "LVA": "AGCTACTAAAGCGTAGTTTTCGTCGTTTGCTGC", # paper implies it behaves similarly to LAA
    "ASV": "AACTGATGCAGCGTAGTTTTCGTCGTTTGCTGC",
}
tags_wt_rc = {name: Seq(seq) for name, seq in tags_wt_rc.items()}
tags_wt = {name: sequence.reverse_complement(seq) for name, seq in tags_wt_rc.items()}

In [ ]:
tags_wt["AAV"].translate()

## Tags

In [ ]:
tag_normalized = {name: workflow.normalize_seq(seq) for name, seq in tags_wt.items()}
tag_prefixes = {
    name: seq[: seq.find(tag_overhangs[0])] for name, seq in tag_normalized.items()
}
tag_prefixes_list = list(tag_prefixes.values())
tag_prefix = tag_prefixes_list[0]
assert all(s == tag_prefix for s in tag_prefixes_list)

In [ ]:
tag_suffix = sequence.smoosh_sequences(workflow.normalize_seq("taa"), tag_overhangs[1])

In [ ]:
head_length_without_prefix = head_length - len(tag_prefix)  # nt
tail_length_with_suffix = tail_length + len(tag_suffix)  # nt

In [ ]:
tag_parts = {name: seq[len(tag_prefix) :] + tag_suffix for name, seq in tags_wt.items()}

In [ ]:
tag_parts

In [ ]:
tags = {}
for name, seq in tag_parts.items():
    tags[f"degtag_{name}"] = {
        "Sequence": seq,
        "Description": f"Wild-type {name} ClpXP degradation tag from Andersen 1998.",
    }
    assert (
        len(seq) - head_length_without_prefix - tail_length_with_suffix
        == nnk_length * 3
    )
    tags[f"degtag_{name}_NNK"] = {
        "Sequence": seq[:head_length_without_prefix]
        + "NNK" * nnk_length
        + seq[-tail_length_with_suffix:],
        "Description": f"ClpXP degradation tag library with 2x alanines, {nnk_length}x NNK's, and the {name} tail from Andersen 1998.",
    }
tags[f"degtag_all_NNK"] = {
    "Sequence": seq[:head_length_without_prefix]
    + "NNK" * nnk_clpx_length
    + seq[-(tail_length_with_suffix - clpx_tag_length * 3) :],
    "Description": f"ClpXP degradation tag library with 2x alanines and {nnk_clpx_length}x NNK's.",
}

In [ ]:
tags

In [ ]:
library_primer_seq = workflow.normalize_seq(reg.get(library_primer)["Sequence"])

In [ ]:
base = {"Author": "Jacob Quinn Shenker", "Date": workflow.date()}

oligo_base = {
    **base,
    "Order date": workflow.date(),
    "Vendor": "IDT",
    "Type": "Primer",
}

part_base = {
    **base,
    "Tags": part_tags,
    "Reference": reference,
}

# apply = {"Sequence": workflow.normalize_seq}
apply = {"Name": None}

for name, row in tags.items():
    # add BsaI flanks
    oligo_seq = (
        design.random_bases(num_random_bases)
        + design.type2s_with_spacer(tag_enzyme, len(tag_overhangs[0]))
        + row["Sequence"]
        + sequence.reverse_complement(
            design.type2s_with_spacer(tag_enzyme, len(tag_overhangs[0]))
        )
    )
    oligo_seq = workflow.normalize_seq(oligo_seq)
    if workflow.DEGENERATE_BASES_REGEX.search(workflow.normalize_seq(row["Sequence"])):
        # is library with degenerate bases, append library primer
        oligo_seq = oligo_seq + library_primer_seq
        oligo_row = {**oligo_base, **row, "Name": name, "Sequence": oligo_seq.upper()}
        oligo_id = olib_oligos.upsert(oligo_row, apply=apply)
        usage = f"{oligo_id}~,{library_primer}/{tag_enzyme}"
    else:
        # not library, just order two oligos to anneal
        # in this case, there's no reverse primer binding site,
        # so we need to add random bases to ensure efficient cutting
        oligo_seq = oligo_seq + design.random_bases(num_random_bases)
        oligo_id = olib_oligos.upsert(
            {
                **oligo_base,
                **row,
                "Name": f"{name}_sense",
                "Sequence": workflow.normalize_seq_upper(oligo_seq),
            },
            apply=apply,
        )
        oligo_id2 = olib_oligos.upsert(
            {
                **oligo_base,
                **row,
                "Name": f"{name}_antisense",
                "Sequence": workflow.normalize_seq_upper(oligo_seq),
            },
            apply=apply,
        )
        usage = f"{oligo_id}={oligo_id2}/{tag_enzyme}"
    part_row = {
        **part_base,
        **row,
        "Sequence": workflow.normalize_seq(row["Sequence"]),
        "Usage": usage,
        "Type": tag_part_type,
        "Upstream overhang": tag_overhangs[0],
        "Downstream overhang": tag_overhangs[1],
        "Species/codon usage": "E. coli",
    }
    lib_parts[name] = part_row
    # lib_parts.upsert(part_row, apply=apply)

In [ ]:
olib_oligos.save()
lib_parts.save()

## Placeholders

In [ ]:
# part_names = ["sigW", "rsiW", "ECF20_992", "AS20_992", "sfGFP"]  # TODO: pick correct FPs
part_names = ["sigW", "rsiW"]
# part_names = ["rsiW"]

In [ ]:
placeholder = tag_prefix + design.golden_gate_placeholder(
    placeholder_enzyme, None, *tag_overhangs
)
# remove downstream overhang because it is already in the storage vector
placeholder = placeholder[: -len(tag_overhangs[1])]

In [ ]:
%%time
placeholder_primers = {}
storage_vector_seqs = {}
source_plasmid_names = {}
for part_name in part_names:
    part = reg.get(part_name)
    part_seq = part["_seq"]
    cds_start, cds_stop = workflow.find_coding_sequence(part_seq)
    cds_stop -= 3  # want to insert placeholder before stop codon
    plasmid_name = workflow.get_source_plasmid(reg, part["Usage"])
    source_plasmid_names[part_name] = plasmid_name
    plasmid_seq = reg.get(plasmid_name)["_seq"]
    part_start, part_stop, _, _ = sequence.find_subsequence(
        plasmid_seq, part_seq, min_score=len(part_seq)
    )
    start = part_start + cds_start
    stop = part_stop - (len(part_seq) - cds_stop)
    downstream_overhang_start = part_stop - len(part["Downstream overhang"])
    # we want 5' end of forward primer binding site to start with the downstream overhang
    # ("aggt" for CDS parts)
    plasmid_seq_forward = workflow.normalize_seq(
        plasmid_seq.reindex(downstream_overhang_start)
    )
    plasmid_seq_reverse = workflow.normalize_seq(
        plasmid_seq.reindex(stop).reverse_complement()
    )
    homology = next(
        primers.iter_primers(
            plasmid_seq_forward,
            min_tm=tm_homology,
            min_mfe=min_mfe,
            anchor="5prime",
            gc_clamp=False,
        )
    )
    forward_primer = next(
        primers.iter_primers(
            plasmid_seq_forward,
            min_length=len(homology),
            min_tm=tm_binding,
            min_mfe=min_mfe,
            anchor="5prime",
        )
    )
    overhang = sequence.reverse_complement(placeholder + homology.binding)
    reverse_primer = next(
        primers.iter_primers(
            plasmid_seq_reverse,
            overhang=overhang,
            min_tm=tm_binding,
            min_mfe=min_mfe,
            anchor="5prime",
        )
    )
    primer_pair = primers.PrimerPair(forward_primer, reverse_primer)
    placeholder_primers[part_name] = primer_pair
    storage_vector_seqs[part_name] = sequence.pcr(
        plasmid_seq, str(primer_pair.primer1), str(primer_pair.primer2)
    ).assemble(method="gibson")

### Oligos

In [ ]:
placeholder_base = {"Author": "Jacob Quinn Shenker", "Date": workflow.date()}

placeholder_oligo_base = {
    **placeholder_base,
    "Order date": workflow.date(),
    "Vendor": "IDT",
    "Type": "Primer",
}

apply = {"Sequence": workflow.normalize_seq}
placeholder_oligo_names = {}

for part_name in part_names:
    name = f"degtag_ph_{part_name}"
    description = f"Adds a deg tag placeholder to {source_plasmid_names[part_name]} ({part_name})."
    forward_primer_id = olt_oligos.upsert(
        {
            **placeholder_oligo_base,
            "Name": f"{name}_f",
            "Sequence": workflow.normalize_seq_upper(
                placeholder_primers[part_name].primer1
            ),
            "Description": description,
        },
        apply=apply,
    )
    reverse_primer_id = olt_oligos.upsert(
        {
            **placeholder_oligo_base,
            "Name": f"{name}_r",
            "Sequence": workflow.normalize_seq_upper(
                placeholder_primers[part_name].primer2
            ),
            "Description": description,
        },
        apply=apply,
    )
    placeholder_oligo_names[part_name] = (forward_primer_id, reverse_primer_id)

In [ ]:
olt_oligos.local

### Storage vectors/parts

In [ ]:
reg.get("sigW")

In [ ]:
frags = enzyme.re_digest(s, Restriction.BsaI)

In [ ]:
frags

In [ ]:
print(frags[0])

In [ ]:
print(frags[1])

In [ ]:
print(frags[2])

In [ ]:
print(workflow.re_digest_part(reg.get("pLIB214")["_seq"], Restriction.BsaI))

In [ ]:
print(workflow.re_digest_part(s, Restriction.BsaI))

In [ ]:
placeholder_part_base = {
    **placeholder_base,
    #    "Tags": part_tags, # ADD TAGS
}

apply = {"Name": None}

for part_name in part_names:
    #### PART SEQUENCE
    #### USAGE
    name = f"{part_name}_degtag_ph"
    part = reg.get(part_name)
    # usage = f"{oligo_id}={oligo_id2}/{part_enzyme}"
    part_row = {
        **placeholder_part_base,
        "Sequence": workflow.normalize_seq(row["Sequence"]),
        "Usage": usage,
        "Type": part["Type"],
        "Upstream overhang": tag_overhangs[0],
        "Downstream overhang": tag_overhangs[1],
        "Species/codon usage": "E. coli",
    }
    lib_parts[name] = part_row
    # lib_parts.upsert(part_row, apply=apply)

In [ ]:
olt.upsert()

In [ ]:
# storage vector maps
# storage vector plasmid entry
# storage vector strain entry

In [ ]:
# olib_oligos.save()
olt_oligos.save()
# lib_parts.save()